In [3]:
import numpy as np
import pandas as pd
import json
import pprint
import json
pp = pprint.PrettyPrinter(indent=4)
pd.options.display.max_colwidth = 100

In [4]:
def df_mean(path, apply_to_df):
    df_mean = pd.read_csv(path)
    df_mean = apply_to_df(df_mean)
        
    if 'nb_movies_not_in_train' in df_mean:
        df_mean = df_mean.drop(['nb_movies_not_in_train'],axis=1)
    if 'nb_test_ratings' in df_mean:
        df_mean = df_mean.drop(['nb_test_ratings'],axis=1)
    if 'nb_train_ratings' in df_mean:
        df_mean = df_mean.drop(['nb_train_ratings'],axis=1)
    if 'user_id' in df_mean:
        df_mean = df_mean.drop(['user_id'],axis=1)
    
    df_mean = df_mean.mean(0).to_frame().T
    df_mean['path'] = path
    
    return df_mean
    

def get_df_results(results, max_train_ratings=None):
    if not max_train_ratings:
        apply_to_df = lambda df: df[df['nb_test_ratings'] > 0]
    else:
        apply_to_df = lambda df: df[(df['nb_test_ratings'] > 0) & (df['nb_train_ratings'] <= max_train_ratings)]
        
    result_map = map(lambda path: df_mean(path, apply_to_df), results)
    return reduce(lambda x, y: x.append(y), result_map)

## BEST ML-100k - 0.7 train / 0.3 test - sparse item

In [17]:
user_metrics = ['../metrics/mfnn/2016-06-20_22.08.26_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv']
df_results = get_df_results(user_metrics)
df_results.sort_values('auc', ascending=False)

,auc,avg_precision,f1,fcp,precision_at_20,recall_at_20,reciprocal_rank,spearman_rank_corr,path
0,0.933496,0.306996,0.2696,0.715244,0.266702,0.415502,0.637832,0.190412,../metrics/mfnn/2016-06-20_22.08.26_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv


In [18]:
# best config
with open('../models/mfnn/2016-06-20_22.08.26_mfnn_ml-100k_e20_tt-0.7_task-3_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'ada_eps': 1e-06,
    u'adagrad': True,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'd2v_model': u'doc2vec-models/2016-04-14_17.36.08_20e_pv-dbow_size50_lr0.025_window8_neg5',
    u'eval_in_parallel': True,
    u'experiment_name': u'mfnn_ml-100k_e20_tt-0.7_task-3',
    u'hit_threshold': 4,
    u'lr': 0.015,
    u'lr_decay': 0.02,
    u'metrics_save_dir': u'metrics/mfnn',
    u'model_save_dir': u'models/mfnn',
    u'nb_d2v_features': 50,
    u'nb_epochs': 20,
    u'nb_latent_f': 128,
    u'nb_movies': 1261,
    u'nb_users': 943,
    u'pool_size': 24,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-100k/ratings.csv',
    u'reg_lambda': 0.01,
    u'run_eval': True,
    u'run_movie_metrics': True,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-100k/sparse-item/0.7-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-100k/sparse-item/0.7-train.

In [19]:
movie_metrics = ['../metrics/mfnn/2016-06-20_22.08.26_mfnn_ml-100k_e20_tt-0.7_task-3_movie-metrics.csv']
df_results = get_df_results(movie_metrics)
df_results.sort_values('auc', ascending=False)

,auc,movie_id,nb_times_in_top_n_predictions,path
0,0.905217,708.548841,78.062914,../metrics/mfnn/2016-06-20_22.08.26_mfnn_ml-100k_e20_tt-0.7_task-3_movie-metrics.csv


## BEST ML-1m - 0.7 train / 0.3 test - sparse item

In [ ]:
user_metrics = ['../metrics/mfnn/2016-05-27_20.57.43_no-si_ml-1m_e20_tt-0.7_task-22_user-metrics.csv']
df_results = get_df_results(user_metrics)
df_results.sort_values('auc', ascending=False)

In [ ]:
# best config
with open('../models/mfnn/2016-06-20_18.30.50_no-si_ml-100k_e20_tt-0.7_base_config.json') as f:
    pp.pprint(json.loads(f.read()))

In [ ]:
movie_metrics = ['../metrics/mfnn/2016-06-20_18.30.50_no-si_ml-100k_e20_tt-0.7_base_movie-metrics.csv']
df_results = get_df_results(movie_metrics)
df_results.sort_values('auc', ascending=False)

# Gridsearch

## ML-100k - 0.7 train / 0.3 test - sparse item

In [15]:
user_metrics = [
'../metrics/mfnn/2016-06-20_14.41.10_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv',
'../metrics/mfnn/2016-06-20_15.23.33_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv',
'../metrics/mfnn/2016-06-20_15.27.24_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv',
'../metrics/mfnn/2016-06-20_15.35.53_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv',
'../metrics/mfnn/2016-06-20_15.57.48_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv',
'../metrics/mfnn/2016-06-20_16.59.18_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv',
'../metrics/mfnn/2016-06-20_17.56.21_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv',
'../metrics/mfnn/2016-06-20_18.03.09_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv',
'../metrics/mfnn/2016-06-20_19.02.32_mfnn_ml-100k_e20_tt-0.7_task-2_user-metrics.csv',
'../metrics/mfnn/2016-06-20_20.07.00_mfnn_ml-100k_e20_tt-0.7_task-2_user-metrics.csv',
'../metrics/mfnn/2016-06-20_20.14.07_mfnn_ml-100k_e20_tt-0.7_task-2_user-metrics.csv',
'../metrics/mfnn/2016-06-20_22.08.26_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv',
'../metrics/mfnn/2016-06-20_22.35.49_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv',
'../metrics/mfnn/2016-06-21_03.33.28_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv',
'../metrics/mfnn/2016-06-21_04.38.48_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv',
'../metrics/mfnn/2016-06-21_06.21.30_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv',
'../metrics/mfnn/2016-06-21_08.17.14_mfnn_ml-100k_e20_tt-0.7_task-2_user-metrics.csv',
'../metrics/mfnn/2016-06-21_09.15.54_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv',
'../metrics/mfnn/2016-06-21_10.20.59_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv',
'../metrics/mfnn/2016-06-21_13.27.13_mfnn_ml-100k_e20_tt-0.7_task-2_user-metrics.csv',
'../metrics/mfnn/2016-06-21_15.30.47_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv',

]

In [16]:
df_results = get_df_results(user_metrics)
df_results.sort_values('auc', ascending=False)

,auc,avg_precision,f1,fcp,precision_at_20,recall_at_20,reciprocal_rank,spearman_rank_corr,path
0,0.933496,0.306996,0.269600,0.715244,0.266702,0.415502,0.637832,0.190412,../metrics/mfnn/2016-06-20_22.08.26_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv
0,0.929596,0.282412,0.250763,0.714033,0.249099,0.385689,0.612299,0.186699,../metrics/mfnn/2016-06-20_16.59.18_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv
0,0.929300,0.351118,0.294186,0.745600,0.292100,0.450057,0.702588,0.280343,../metrics/mfnn/2016-06-21_03.33.28_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv
0,0.928658,0.278104,0.249006,0.713842,0.247720,0.382489,0.605962,0.187090,../metrics/mfnn/2016-06-20_15.23.33_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv
0,0.928464,0.353380,0.292917,0.746260,0.291039,0.446157,0.707856,0.282339,../metrics/mfnn/2016-06-21_08.17.14_mfnn_ml-100k_e20_tt-0.7_task-2_user-metrics.csv
0,0.928007,0.320823,0.273305,0.744593,0.272747,0.414704,0.675805,0.277141,../metrics/mfnn/2016-06-20_15.27.24_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv
0,0.926705,0.314472,0.268958,0.744357,0.268346,0.408378,0.666743,0.276841,../metrics/mfnn/2016-06-21_06.21.30_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv
0,0.925691,0.303821,0.260660,0.743365,0.259650,0.398210,0.654882,0.273701,../metrics/mfnn/2016-06-20_22.35.49_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv
0,0.924101,0.295554,0.256256,0.741017,0.255037,0.392301,0.638360,0.266473,../metrics/mfnn/2016-06-21_13.27.13_mfnn_ml-100k_e20_tt-0.7_task-2_user-metrics.csv
0,0.920026,0.254347,0.233028,0.712977,0.235207,0.349612,0.581458,0.183622,../metrics/mfnn/2016-06-21_10.20.59_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv


In [14]:
# best config
with open('../models/mfnn/2016-06-20_22.08.26_mfnn_ml-100k_e20_tt-0.7_task-3_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'ada_eps': 1e-06,
    u'adagrad': True,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'd2v_model': u'doc2vec-models/2016-04-14_17.36.08_20e_pv-dbow_size50_lr0.025_window8_neg5',
    u'eval_in_parallel': True,
    u'experiment_name': u'mfnn_ml-100k_e20_tt-0.7_task-3',
    u'hit_threshold': 4,
    u'lr': 0.015,
    u'lr_decay': 0.02,
    u'metrics_save_dir': u'metrics/mfnn',
    u'model_save_dir': u'models/mfnn',
    u'nb_d2v_features': 50,
    u'nb_epochs': 20,
    u'nb_latent_f': 128,
    u'nb_movies': 1261,
    u'nb_users': 943,
    u'pool_size': 24,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-100k/ratings.csv',
    u'reg_lambda': 0.01,
    u'run_eval': True,
    u'run_movie_metrics': True,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-100k/sparse-item/0.7-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-100k/sparse-item/0.7-train.

## ML-1M - 0.7 train / 0.3 test - sparse item

In [ ]:
results = [

]

In [ ]:
result_map = map(lambda p: (p, pd.read_csv(p)['auc'].mean()), results)
sorted(result_map, key=lambda t:t[1], reverse=True)

In [ ]:
# best config
with open('../models/mfnn/_config.json') as f:
    pp.pprint(json.loads(f.read()))

In [ ]:
best_metrics = pd.read_csv('../metrics/mfnn/_user-metrics.csv')
best_metrics.mean(0)